神经网络的学习

- 对于线性可分问题，第2章的感知机是可以利用数据自动学习的。
根据“感知机收敛定理”，通过有限次数的学习，线性可分问题是可
解的。但是，非线性可分问题则无法通过（自动）学习来解决

- 深 度 学 习 有 时 也 称 为 端 到 端 机 器 学 习（end-to-end machine learning）。这里所说的端到端是指从一端到另一端的意思，也就是
从原始数据（输入）中获得目标结果（输出）的意思。

- 机器学习中，一般将数据分为训练数据和测试数据两部分来进行学习和
实验等。首先，使用训练数据进行学习，寻找最优的参数；然后，使用测试
数据评价训练得到的模型的实际能力。为什么需要将数据分为训练数据和测
试数据呢？因为我们追求的是模型的泛化能力。为了正确评价模型的泛化能
力，就必须划分训练数据和测试数据。另外，训练数据也可以称为监督数据。

- 泛化能力是指处理未被观察过的数据（不包含在训练数据中的数据）的
能力。获得泛化能力是机器学习的最终目标

损失函数

- 损失函数是表示神经网络性能的“恶劣程度”的指标，即当前的
神经网络对监督数据在多大程度上不拟合，在多大程度上不一致。
以“性能的恶劣程度”为指标可能会使人感到不太自然，但是如
果给损失函数乘上一个负值，就可以解释为“在多大程度上不坏”，
即“性能有多好”。并且，“使性能的恶劣程度达到最小”和“使性
能的优良程度达到最大”是等价的，不管是用“恶劣程度”还是“优
良程度”，做的事情本质上都是一样的。

均方误差(mean squared error)

- $E=\dfrac{1}{2}\sum\limits_k(y_k-t_k)^2$

- $y_k$表示神将网络的输出，$t_k$表示监督数据，$k$表示数据的维数

- 将正确解标签表示为1，其他标签表示为0的表示方法称为**one-hot表示**

In [3]:
import numpy as np

def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

# 设“2”为正确解
t = [0,0,1,0,0,0,0,0,0,0]

# 例1：“2“的概率最高的情况(0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(mean_squared_error(np.array(y),np.array(t)))

# 例2：“7“的概率最高的情况(0.6)
z = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(mean_squared_error(np.array(z),np.array(t)))

0.09750000000000003
0.5975


交叉熵误差

- $E=-\sum\limits_kt_k\log y_k$

- $y_k$是神经网络的输出，$t_k$是正确解标签

- $t_k$中只有正确解标签的索引为1，其他均为0(one-hot表示)

- 上式实际上只计算对应正确解标签的输出的自然对数

In [ ]:
def cross_entropy_error(y, t):
    delta = 1e-7    #加上一个微小值delta，防止负无穷大的发生
    return -np.sum(t*np.log(y + delta))

mini-batch学习

- mini-batch的损失函数利用一部分样本数据来近似地计算整体。也就是说，用随机选择的小批量数据(mini-batch)作为全体训练数据的近似值

为何要设定损失函数？

- 在进行神经网路的学习时，不能将识别精度作为指标。因为如果以识别精度为指标，则参数的导数在绝大多数地方都会变为0

数值微分

In [ ]:
# 不好的实现示例
def numerical_diff(f, x):
    h = 10e-50
    return (f(x + h) - f(x)) / h    # 前向差分

- 函数numerical_diff(f, x)的名称来源于**数值微分**的英文numerical differentiation

- 在上面的实现中，因为想把尽可能小的值赋给h（可以的话，想让h无线接近0），所以h实用了10e-50这个微小值。但是这样反而产生了**舍入误差**(rounding error)。所谓舍入误差，是指因省略小数的精细部分的数值（比如，小数点第8位以后的数值）而造成最终的计算结果上的误差。

In [ ]:
def numerical_diff(f, x):
    h = 1e-4    # 0.0001
    return (f(x+h) - f(x-h)) / (2*h)    #中心差分
#中心差分算法以x为中心，计算它左右两边的差分

梯度法

In [7]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x

    for i in range(step_num):
        grad = numerical_gradient(f,x)
        x -= lr * grad
    
    return x

- 参数f是要进行最优化的函数，init_x是初始值，lr是学习率learning 
rate，step_num是梯度法的重复次数。numerical_gradient(f,x)会求函数的
梯度，用该梯度乘以学习率得到的值进行更新操作，由step_num指定重复的
次数。

epoch

- epoch是一个单位。一个epoch表示学习中所有训练数据均被使用过一次时的更新次数。比如，对于10000笔训练数据，用大小为100笔数据的mini-batch进行学习时，重复随机梯度下降法100次，所有的训练数据就都被“看过”了。此时，100次就是一个epoch

- 实际上，一般做法是事先将所有训练数据随机打乱，然后按指定的批次大小，按序生成mini-batch。这样每个mini-batch均有一个索引号，比如此例可以是0,1,2,...,99，然后用索引号可以遍历所有的mini-batch。遍历一次所有数据，就称为一个epoch